In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import statistics
import math

In [502]:
data = pd.read_csv("s3://ai-diennea/data/export_wonkit_20210630102441.csv.gz")
# add the day of the week column
day_of_week = []
for i in range(len(data)):
    day_of_week.append(pd.to_datetime(data["EVENT.DATE"][i], format='%Y/%m/%d %H:%M').day_name())
data['day_of_week'] = pd.DataFrame(day_of_week)

/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3072: DtypeWarning: Columns (8) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [503]:
len(data)
data = data.iloc[:10000, :]

In [576]:
def sent_date_to_min(sents):
    return sents.hour*60 + sents.minute

def exp_decay_fit(x, sent_open_hour_range):
    if x > sent_open_hour*60:
        return .0
    return math.exp(((-math.log(2)/(sent_open_hour*60))*x) + math.log(2))-1


def compute_fitSA(hash_mex, hash_contact, sent_open_hour_range, data):
    sents, opens, clicks = [], [], []
    
    for i in range(len(data)):
        if data['HashMessaggio'][i] == hash_mex and data['HashContatto'][i] == hash_contact:
            if data['EVENT.TYPE'][i] == 'Sent':
                sents.append(pd.to_datetime(data["EVENT.DATE"][i], format='%Y/%m/%d %H:%M'))
            elif data['EVENT.TYPE'][i] == 'Open':
                opens.append(pd.to_datetime(data["EVENT.DATE"][i], format='%Y/%m/%d %H:%M'))
            elif data['EVENT.TYPE'][i] == 'Click':
                clicks.append(pd.to_datetime(data["EVENT.DATE"][i], format='%Y/%m/%d %H:%M'))
    y = sent_date_to_min(sents[0]) # since this function gives me for free the y, I return it here
    # if there are multiple opens/clicks, take the oldest one (the first that has been opened/clicked)
    oldest = None
    if opens != []:
        for i in opens:
            if oldest is None:
                oldest = i
            elif i < oldest:
                oldest = i
    elif clicks != []:
        for i in clicks:
            if oldest is None:
                oldest = i
            elif i < oldest:
                oldest = i
    else: # this means that the mail has never been opened/clicked
        return .0, y

    # compute minutes of the distance between sent-open/sent-click
    mins = ((oldest - sents[0]).days*24*60) + ((oldest - sents[0]).seconds//3600)*60 + ((oldest - sents[0]).seconds//60)%60
    
    return exp_decay_fit(mins, sent_open_hour_range), y

def compute_fitAC(hash_mex, hash_contact, open_click_hour_range, data):
    opens, clicks = [], []

    for i in range(len(data)):
        if data['HashMessaggio'][i] == hash_mex and data['HashContatto'][i] == hash_contact:
            if data['EVENT.TYPE'][i] == 'Open':
                opens.append(pd.to_datetime(data["EVENT.DATE"][i], format='%Y/%m/%d %H:%M'))
            elif data['EVENT.TYPE'][i] == 'Click':
                clicks.append(pd.to_datetime(data["EVENT.DATE"][i], format='%Y/%m/%d %H:%M'))

    oldest_open = None
    oldest_click = None

    if opens == [] and clicks != []: # messege clicked but open is not detected: I assign -1 and then a function
        return -1                    # computes the avg
    
    if clicks == []: # covers cases when a message is sent and is never open and never clicked
        return .0
    elif opens != [] and clicks != []:
        for i in opens: # get oldest open
            if oldest_open is None:
                oldest_open = i
            elif i < oldest_open:
                oldest_open = i
        for i in clicks: # get oldest click
            if oldest_click is None:
                oldest_click = i
            elif i < oldest_click:
                oldest_click = i
    # compute minutes of the distance between sent-open/sent-click
    mins = ((oldest_click - oldest_open).days*24*60) + ((oldest_click - oldest_open).seconds//3600)*60 + ((oldest_click - oldest_open).seconds//60)%60
    return exp_decay_fit(mins, open_click_hour_range)

def avg_fitAC(df):
    fitAC_column = df['fitAC'].to_list()
    l = []
    for i in fitAC_column:
        if i != -1:
            l.append(i)
    return statistics.mean(l)

In [554]:
# (messaggi aperti fascia oraria 0-1 / numero totali di mail aperte in generale dal contatto)
def open_rate_mex_time_slot(hash_contact, data): # TODO: add parameter for [0, 24]
    mex_plus_contact_distinct = []
    dates = []
    for i in range(len(data)):
        if "nan" not in str(data["EVENT.DATE"][i]) and data['HashContatto'][i] == hash_contact and str(data["EVENT.TYPE"][i]) == 'Open' and (data["HashMessaggio"][i] + data['HashContatto'][i]) not in mex_plus_contact_distinct:
            mex_plus_contact_distinct.append(data["HashMessaggio"][i] + data['HashContatto'][i])
            dates.append(pd.to_datetime(data["EVENT.DATE"][i], format='%Y/%m/%d %H:%M'))
    # add messages clicked but not opened
    for i in range(len(data)):
        if "nan" not in str(data["EVENT.DATE"][i]) and data['HashContatto'][i] == hash_contact and str(data["EVENT.TYPE"][i]) == 'Click' and (data["HashMessaggio"][i] + data['HashContatto'][i]) not in mex_plus_contact_distinct:
            mex_plus_contact_distinct.append(data["HashMessaggio"][i] + data['HashContatto'][i])
            dates.append(pd.to_datetime(data["EVENT.DATE"][i], format='%Y/%m/%d %H:%M'))

    tot_mex_opened = len(mex_plus_contact_distinct)
    if len(dates) == 0: # it means that the user has never opened messages
        return np.zeros((24,))
    df = pd.DataFrame(dates, columns =['dates'])

    df = df.assign(session=pd.cut(df["dates"].dt.hour,
                                [0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24],
                                labels=['0-1','1-2','2-3','3-4','4-5','5-6','6-7', '7-8', '8-9', '9-10', '10-11', '11-12', '12-13', '13-14', '14-15', '15-16', '16-17', '17-18', '18-19', '19-20', '20-21', '21-22', '22-23', '23-24'],
                                include_lowest=True))
    numpy_output = df['session'].value_counts().sort_index().to_numpy() / tot_mex_opened

    return numpy_output


def dp_open_rate_mex_time_slot(data):
    d = {}
    for i in data["HashContatto"].unique():
        if i not in d:
            d[i] = None
    for key in d:
        d[key] = open_rate_mex_time_slot(key, data)

    return d


# (messaggi clickati fascia oraria 0-1 / numero totali di mail clickate in generale)
def click_rate_mex_time_slot(hash_contact, data): # TODO: add parameter for [0, 24]
    mex_plus_contact_distinct = []
    dates = []
    for i in range(len(data)):
        if "nan" not in str(data["EVENT.DATE"][i]) and data['HashContatto'][i] == hash_contact and str(data["EVENT.TYPE"][i]) == 'Click' and (data["HashMessaggio"][i] + data['HashContatto'][i]) not in mex_plus_contact_distinct:
            mex_plus_contact_distinct.append(data["HashMessaggio"][i] + data['HashContatto'][i])
            dates.append(pd.to_datetime(data["EVENT.DATE"][i], format='%Y/%m/%d %H:%M'))

    tot_mex_opened = len(mex_plus_contact_distinct)
    if len(dates) == 0: # it means that the user has never clicked messages
        return np.zeros((24,))
    df = pd.DataFrame(dates, columns =['dates'])

    df = df.assign(session=pd.cut(df["dates"].dt.hour,
                                [0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24],
                                labels=['0-1','1-2','2-3','3-4','4-5','5-6','6-7', '7-8', '8-9', '9-10', '10-11', '11-12', '12-13', '13-14', '14-15', '15-16', '16-17', '17-18', '18-19', '19-20', '20-21', '21-22', '22-23', '23-24'],
                                include_lowest=True))
    

    numpy_output = df['session'].value_counts().sort_index().to_numpy() / tot_mex_opened
    return numpy_output

def dp_click_rate_mex_time_slot(data):
    d = {}
    for i in data["HashContatto"].unique():
        if i not in d:
            d[i] = None
    for key in d:
        d[key] = click_rate_mex_time_slot(key, data)

    return d

In [577]:
# numero di messaggi aperti tra le 0-1 di quella Comm / Quanti messaggi distinti sono stati aperti/clickati in quella comunicazione
# return a dict where in `d[key]['time_slot_distribution']` there is the 24 vector time slot distribution
def rate_comm_time_slot(data, open_or_click):
    if open_or_click != 'Open' and open_or_click != 'Click':
        raise Exception('open_or_click value must be either \'Open\' or \'Click\'') 
    d = {}
    for i in data["COMUNICATION.COMUNICATION_NAME"].unique():
        if i not in d:
            d[i] = {'tot_open': 0, 'dates': [], 'time_slot_distribution': None}

    for comm in data['COMUNICATION.COMUNICATION_NAME'].unique():
        mex_plus_contact_distinct = []
        for i in range(len(data)):
            if data['COMUNICATION.COMUNICATION_NAME'][i] == comm and str(data["EVENT.TYPE"][i]) == open_or_click and (data["HashMessaggio"][i] + data['HashContatto'][i]) not in mex_plus_contact_distinct:
                mex_plus_contact_distinct.append(data["HashMessaggio"][i] + data['HashContatto'][i])
                d[data['COMUNICATION.COMUNICATION_NAME'][i]]['tot_open'] += 1
                d[data['COMUNICATION.COMUNICATION_NAME'][i]]['dates'].append(pd.to_datetime(data["EVENT.DATE"][i], format='%Y/%m/%d %H:%M'))
    for key, value in d.items():
        if len(d[key]['dates']) == 0: # it means that the communcation has never been opened
            return np.zeros((24,))
        df = pd.DataFrame(d[key]['dates'], columns =['dates'])

        df = df.assign(session=pd.cut(df["dates"].dt.hour,
                                    [0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24],
                                    labels=['0-1','1-2','2-3','3-4','4-5','5-6','6-7', '7-8', '8-9', '9-10', '10-11', '11-12', '12-13', '13-14', '14-15', '15-16', '16-17', '17-18', '18-19', '19-20', '20-21', '21-22', '22-23', '23-24'],
                                    include_lowest=True))

        d[key]['time_slot_distribution'] = df['session'].value_counts().sort_index().to_numpy() / d[key]['tot_open']

    return d

def get_index_for_comm(hash_mex, hash_contact, data):
    return data.index[(data['HashMessaggio'] == hash_mex) & (data['HashContatto'] == hash_contact)][0] # get first index because the others are the same
    

## Create all dataset

OR = open rate <br>
CR = click rate <br>
OR-C = open rate per communication <br>
CR-C = click rate per communication<br>
fitSA = fitness Sent-Apertura <br>
fitAC = fitness Apertura-Click

In [578]:
column_names = ["HashContatto", "OR 0-1", "OR 1-2", "OR 2-3", "OR 3-4", "OR 4-5", "OR 5-6", "OR 6-7", "OR 7-8", "OR 8-9", "OR 9-10", "OR 10-11", "OR 11-12",
"OR 12-13", "OR 13-14", "OR 14-15", "OR 15-16", "OR 16-17", "OR 17-18", "OR 18-19", "OR 19-20", "OR 20-21", "OR 21-22", "OR 22-23", "OR 23-24",

"CR 0-1", "CR 1-2", "CR 2-3", "CR 3-4", "CR 4-5", "CR 5-6", "CR 6-7", "CR 7-8", "CR 8-9", "CR 9-10", "CR 10-11", "CR 11-12",
"CR 12-13", "CR 13-14", "CR 14-15", "CR 15-16", "CR 16-17", "CR 17-18", "CR 18-19", "CR 19-20", "CR 20-21", "CR 21-22", "CR 22-23", "CR 23-24",

"OR-C 0-1", "OR-C 1-2", "OR-C 2-3", "OR-C 3-4", "OR-C 4-5", "OR-C 5-6", "OR-C 6-7", "OR-C 7-8", "OR-C 8-9", "OR-C 9-10", "OR-C 10-11", "OR-C 11-12", 
"OR-C 12-13", "OR-C 13-14", "OR-C 14-15", "OR-C 15-16", "OR-C 16-17", "OR-C 17-18", "OR-C 18-19", "OR-C 19-20", "OR-C 20-21", "OR-C 21-22", "OR-C 22-23", "OR-C 23-24",

"CR-C 0-1", "CR-C 1-2", "CR-C 2-3", "CR-C 3-4", "CR-C 4-5", "CR-C 5-6", "CR-C 6-7", "CR-C 7-8", "CR-C 8-9", "CR-C 9-10", "CR-C 10-11", "CR-C 11-12", 
"CR-C 12-13", "CR-C 13-14", "CR-C 14-15", "CR-C 15-16", "CR-C 16-17", "CR-C 17-18", "CR-C 18-19", "CR-C 19-20", "CR-C 20-21", "CR-C 21-22", "CR-C 22-23", "CR-C 23-24",
"fitSA", "fitAC", "Label"]

df = pd.DataFrame(columns = column_names)

In [579]:
# 99 -> label

In [581]:
df = pd.DataFrame(0, index=np.arange((len(data['HashContatto'].unique()) + len(data['HashMessaggio'].unique()))), columns=column_names) 

In [582]:
open_rate_comm_dict = rate_comm_time_slot(data, 'Open')
click_rate_comm_dict = rate_comm_time_slot(data, 'Click')

In [583]:
dk = {}
for cont in data['HashContatto'].unique():
    if cont not in dk:
        dk[cont] = {'HashMessaggio': []}
        
for i in range(len(data)):
    dk[data['HashContatto'][i]]['HashMessaggio'].append(data['HashMessaggio'][i])
    
for key in dk:
    vals = list(set(dk[key]['HashMessaggio']))
    dk[key]['HashMessaggio'] = vals

In [ ]:
sent_open_hour_range = 36
open_click_hour_range = 24
current = 0

for cont in dk:
    for mex in dk[cont]['HashMessaggio']:
        # HashContatto
        df.iloc[current, 0] = cont
        fitSA, y = compute_fitSA(mex, cont, sent_open_hour_range, data)
        # Label "Sent" TODO: convert in minutes and then into [0, 1]
        df.iloc[current, 99] = y
        # fitSA
        df.iloc[current, 97] = fitSA
        # fitAC
        fitAC = compute_fitAC(mex, cont, open_click_hour_range, data)
        df.iloc[current, 98] = fitAC
        # index for compute opens and clicks of communication
        index_comm = get_index_for_comm(hash_mex, hash_contact, data)
        # open rate communication
        df.iloc[current, 49:73] = open_rate_comm_dict[data['COMUNICATION.COMUNICATION_NAME'][index_comm]]['time_slot_distribution']
        # click rate communication
        df.iloc[current, 73:97] = click_rate_comm_dict[data['COMUNICATION.COMUNICATION_NAME'][index_comm]]['time_slot_distribution']
        current += 1
        if current % 1000 == 0:
            print(current)

In [ ]:
df = df.iloc[:current, :]

In [ ]:
# open 24

In [ ]:
d = dp_open_rate_mex_time_slot(data)

In [ ]:
for i in range(df):
     df.iloc[i, 1:25] = d[df.iloc[i, 0]] # df.iloc[i, 0] is HashContatto

In [ ]:
# click 24

In [ ]:
d2 = dp_click_rate_mex_time_slot(data)

In [ ]:
for i in range(len(df)):
     df.iloc[i, 25:49] = d2[df.iloc[i, 0]]

In [ ]:
# clicked but not opened have fitAC as -1, so we assign the average of the fitAC of mex cicked and opened
avgAC = avg_fitAC(df)

for i in range(len(df)):
    if df['fitAC'][i] == -1:
        df['fitAC'][i] = avgAC

In [ ]:
df